<a href="https://colab.research.google.com/github/leoperez55/Machine-Learning-Sales-Predictor-/blob/main/Commercial_Sales_Forecasting_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00


preparing input data

In [3]:
from os.path import exists
import pandas as pd

#Checks to see if the training file is already downloaded and if not directly downloads it from the web
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path) #Read csv into a dataframe

#Change 'store' and 'item' columns to strings 
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)
#Change string date columns to pandas date/time columns
df["date"] = pd.to_datetime(df["date"])

#Since we picked 19,000 data points to smpl we need to sort again (chronolog.)
df.sort_values("date", inplace = True)
df.reset_index(inplace = True, drop =True)
#Shows first 5 rows
df.head() 

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,7,12,26
2,2013-01-01,7,46,27
3,2013-01-01,8,12,54
4,2013-01-01,9,12,35


In [4]:
train = df[df["date"] < "2017-01-01"] #Stores data from 2013-2016 into training data set
test = df[df["date"] >= "2017-01-01"] #Stores data from 2017 & up into testing data set



In [5]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]


In [6]:
from upgini.utils.cv_utils import CVType
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

#Gathers relevant features from outside sources that can be used to enrich our data set
enricher = FeaturesEnricher(
    search_keys = {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,train_target,eval_set=[(test_features, test_target)])

<IPython.core.display.Javascript object>

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=56f74b4e-51df-4f5b-a013-c582208d9fb8
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com



17 relevant feature(s) found with the search keys: ['date']


,,item,0.559548,100.000000,categorical,
,,store,0.194305,100.000000,categorical,
Upgini,Public data,f_weather_date_weather_pca_0_d7e0a1fc,0.089173,100.000000,numerical,Free
Upgini,Public data,f_events_date_year_cos1_9014a856,0.025565,100.000000,numerical,Free
Upgini,Public data,f_events_date_holiday_umap_22_19de2388,0.011485,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_31_fa6d9a99,0.007347,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.006754,100.000000,numerical,Free
Upgini,Public data,f_events_date_holiday_umap_43_a99bb99c,0.003311,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_34_c3ef5b4f,0.003295,100.000000,numerical,Free
Upgini,Public data,f_events_date_holiday_umap_1_e6c92365,0.003016,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_33_89bb7578,0.002511,100.000000,numerical,Free


Calculating accuracy uplift after enrichment...


Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift
,,,,
Train,239719,321.160145,175.647934,145.512210
Eval 1,60281,560.376613,361.485052,198.891561


In [8]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric  #Testing

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)


In [9]:
#Creating enriched versions of our train/test features 
enriched_train_features = enricher.transform(train_features, keep_input = True) #Adds all enriched features found by upgini and keeps all original columns/features (true)
enriched_test_features = enricher.transform(test_features, keep_input = True) #Adds all enriched features found by upgini and keeps all original columns/features (true)
enriched_train_features.head() # Returns top 5 rows of new data set with added features

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=15558563-9aca-4d3c-ae77-f7c31ca8eba3
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


Retrieving selected features from data sources...

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=995964aa-0dda-4046-85d5-a79547dee55c
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


Retrieving selected features from data sources...



,date,store,item,f_weather_date_weather_pca_0_d7e0a1fc,f_events_date_year_cos1_9014a856,f_events_date_holiday_umap_22_19de2388,f_weather_date_weather_umap_31_fa6d9a99,f_events_date_week_sin1_847b5db1,f_events_date_holiday_umap_43_a99bb99c,f_weather_date_weather_umap_34_c3ef5b4f,f_events_date_holiday_umap_1_e6c92365,f_weather_date_weather_umap_33_89bb7578,f_events_date_holiday_pca_1_c8996ea7,f_events_date_holiday_pca_10_8d7d28dd,f_financial_date_usd_f4b772a9,f_events_date_holiday_umap_49_76b28f01,f_events_date_holiday_umap_19_bb63f7c2,f_financial_date_snp500_3e044492,f_economic_date_cpi_umap_4_970cc061,f_financial_date_stoxx_dd33e672
0,2013-01-01,1,1,29.676683,0.98522,5.464547,4.712653,0.781831,4.20097,5.664261,6.23285,4.644803,-11.948969,-8.824001,79.769997,4.547708,4.583521,1426.189941,10.153208,278.779999
1,2013-01-01,7,12,29.676683,0.98522,5.464547,4.712653,0.781831,4.20097,5.664261,6.23285,4.644803,-11.948969,-8.824001,79.769997,4.547708,4.583521,1426.189941,10.153208,278.779999
2,2013-01-01,7,46,29.676683,0.98522,5.464547,4.712653,0.781831,4.20097,5.664261,6.23285,4.644803,-11.948969,-8.824001,79.769997,4.547708,4.583521,1426.189941,10.153208,278.779999
3,2013-01-01,8,12,29.676683,0.98522,5.464547,4.712653,0.781831,4.20097,5.664261,6.23285,4.644803,-11.948969,-8.824001,79.769997,4.547708,4.583521,1426.189941,10.153208,278.779999
4,2013-01-01,9,12,29.676683,0.98522,5.464547,4.712653,0.781831,4.20097,5.664261,6.23285,4.644803,-11.948969,-8.824001,79.769997,4.547708,4.583521,1426.189941,10.153208,278.779999


In [10]:
model.fit(train_features,train_target) #Train our model on our original training data sets (No enriched features)
preds = model.predict(test_features) #Predicting our test data set
eval_metric(test_target.values, preds, "SMAPE") #Snape gives us percentage value of errors, comparing predicted value with testTarget Values

[39.4432592360496]

In [11]:
model.fit(enriched_train_features,train_target) #Train our model on our original training data sets (No enriched features)
enriched_preds = model.predict(enriched_test_features) #Predicting our test data set
eval_metric(test_target.values, enriched_preds, "SMAPE") #Snape gives us percentage value of errors, comparing predicted value with testTarget Values

[12.901367566660687]